In [24]:
import json

# read file json data_skill_tensor.json

with open('data/train/total_skill_jds.json', 'r') as f:
    total_skill_jbs = json.load(f)

f.close()

# Filter Data

In [25]:
import json

with open('data/removed_skill.json', 'r') as f:
    removed_skill = json.load(f)

f.close()

In [26]:
for idx, skills in enumerate(total_skill_jbs):
    total_skill_jbs[idx] = [s for s in skills if s not in removed_skill]

In [27]:
# save to train
import json

json_path = 'data/train/total_skill_jds_removed_minout.json'
with open(json_path, 'w') as f:
    json.dump(total_skill_jbs, f, indent=4)

# Augmentation Data

## Pre Augmentation

In [28]:
# valid data
valid_data_path = 'data/valid/skills_valid.json'

def load_valid_data(filename):

    with open(filename, 'r') as f:
        valid_data = json.load(f)

    f.close()

    # total valid skill
    total_valid_skill = 0
    for skill in valid_data.keys():
        total_valid_skill += len(valid_data[skill])

    print(f'Total valid skill: {total_valid_skill}')

    return valid_data

valid_data = load_valid_data(valid_data_path)

Total valid skill: 36806


In [ ]:
def find_frequency_skill_relevant(skill, k=-1, size=3, valid_data=valid_data):
    frequency = {}
    

    # sort frequency
    frequency = {k: v for k, v in sorted(frequency.items(), key=lambda item: item[1], reverse=True)}

    if k == -1:
        return frequency
    else:
        # return top k skill and remove top 1 (skill a)
        return {i: frequency[i] for i in list(frequency)[1:k+1]}

In [ ]:
def cluster_skill(skills, valid_data=valid_data):
    
    for skill in skills:
        

## Function Augmentation

In [29]:
import random
import numpy as np

def shuffle_data(skills):
    # Make a copy of the input list to avoid modifying the original list
    shuffled_skills = skills.copy()
    
    # Shuffle the copy of the list in place
    random.shuffle(shuffled_skills)
    
    return shuffled_skills

def cutout_data(skills, mask_size=5):

    cutout_skills = skills.copy()
    
    mask_size = min(mask_size, len(cutout_skills))
    
    cutout_index = random.sample(range(len(skills)), mask_size)
    
    for idx in cutout_index:
        cutout_skills[idx] = ''
        

    cutout_skills = [x for x in cutout_skills if x != '']
    
    return cutout_skills


In [30]:
def create_new_data(total_skills, num_samples=2100):

    if num_samples > len(total_skills):
        num_samples = len(total_skills) 

    skills_augmentation_collection = []
    # pick random sample in skills jbs
    random_indexs = random.sample(range(len(total_skills)), num_samples)
    for index in random_indexs:
        mask_size = 0

        augment_skill = shuffle_data(total_skills[index])
        # augment_skill = cutout_data(augment_skill, mask_size=random.randint(0, int(len(total_skills[index])/2)))
        skills_augmentation_collection.append(augment_skill)

    return skills_augmentation_collection

In [31]:
total_augmentation = create_new_data(total_skill_jbs, len(total_skill_jbs)) + create_new_data(total_skill_jbs, len(total_skill_jbs))
total_skill_jbs += total_augmentation

In [32]:
len(total_skill_jbs)

80043

In [33]:
# save to train
import json

json_path = 'data/train/total_skill_augmentation_jds.json'
with open(json_path, 'w') as f:
    json.dump(total_skill_jbs, f, indent=4)

# End